In [41]:
import os
import pickle
import random
import socket
import struct

import networkx as nx
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from src.dataset.dataset_info import datasets
from src.graph.graph_construction import create_weightless_window_graph
from src.graph.graph_measures import calculate_graph_measures
from src.graph.centralities import add_centralities, add_centralities_as_node_features
from local_variables import local_datasets_path


In [42]:
multi_class = True

use_node_features = False

use_port_in_address = False

generated_ips = False

graph_type = "flow"
# graph_type = "window"
# graph_type = "line"

window_size= 500

sort_timestamp = False

# k_fold = None
# k_fold = 5

validation_size = 0.1
test_size = 0.1

cn_measures = ["betweenness", "degree", "pagerank", "closeness", "k_truss"]
# cn_measures = ["betweenness", "degree", "closeness"]

network_features = ['src_betweenness', 'dst_betweenness', 'src_degree', 'dst_degree', 'src_pagerank', 'dst_pagerank', 'src_closeness', 'dst_closeness', 'src_k_truss', 'dst_k_truss']
# network_features = ['src_betweenness', 'dst_betweenness', 'src_degree', 'dst_degree', 'src_pagerank', 'dst_pagerank']

In [43]:
# name = "cic_ton_iot_5_percent"
# name = "cic_ton_iot"
# name = "cic_ids_2017_5_percent"
# name = "cic_ids_2017"
# name = "cic_bot_iot"
# name = "cic_ton_iot_modified"
# name = "nf_ton_iotv2_modified"
# name = "ccd_inid_modified"
# name = "nf_uq_nids_modified"
# name = "edge_iiot"
# name = "nf_cse_cic_ids2018"
# name = "nf_bot_iotv2"
# name = "nf_uq_nids"
name = "x_iiot"

dataset = datasets[name]

In [44]:
g_type = ""
if graph_type == "flow":
    g_type = "flow"
elif graph_type == "line":
    g_type = f"line_graph_{window_size}"
elif graph_type == "window":
    g_type = f"window_graph_{window_size}"
    
if multi_class:
    g_type += "__multi_class"
    
if use_node_features:
    g_type += "__n_feats"
    
# if k_fold:
#     g_type += f"__{k_fold}_fold"
    
if use_port_in_address:
    g_type += "__ports"
    
if generated_ips:
    g_type += "__generated_ips"
    
if sort_timestamp:
    g_type += "__sorted"
else:
    g_type += "__unsorted"
    
dataset_path = os.path.join(local_datasets_path,name)
folder_path = os.path.join(dataset_path, g_type)
# folder_path = f"datasets/{name}/{g_type}"
folder_path

'C:\\Users\\Administrateur\\Desktop\\datasets\\x_iiot\\flow__multi_class__unsorted'

In [45]:
df = pd.read_parquet(os.path.join(dataset_path, f"{name}.parquet"))

In [46]:
df.head()

,Timestamp,Scr_IP,Scr_port,Des_IP,Des_port,Protocol,Service,Duration,Scr_bytes,Des_bytes,...,Login_attempt,Succesful_login,File_activity,Process_activity,read_write_physical.process,is_privileged,class1,class2,class3,Class
index,,,,,,,,,,,,,,,,,,,,,
0,2020-01-09 03:35:56,192.168.2.199,49278,192.168.2.10,80,1,4,75756,1849,13843,...,0,0,0,0,0,0,Scanning_vulnerability,Reconnaissance,1,6
1,2020-01-12 23:31:13,10.0.1.5,39769,131.236.3.92,53,2,2,83,16696,1,...,0,0,0,0,0,0,Normal,Normal,0,4
2,2020-01-08 22:28:06,172.24.1.80,59050,172.24.1.1,53,2,2,132,10806,14586,...,0,0,0,0,0,0,Normal,Normal,0,4
3,2020-02-26 22:54:00,192.168.2.196,37966,192.168.2.10,1880,1,16,260329,689,16960,...,1,1,1,1,1,1,Normal,Normal,0,4
4,2019-12-15 23:30:12,172.24.1.80,38233,172.24.1.1,53,2,2,74,0,0,...,0,0,0,0,0,0,Normal,Normal,0,4


In [47]:
cols_to_norm = list(set(list(df.columns))  - set(list([dataset.label_col, dataset.class_num_col])) - set(dataset.drop_columns)  - set(dataset.weak_columns))

In [48]:
df[dataset.label_col].value_counts()

class3
0    324810
1    307004
Name: count, dtype: int64

In [49]:
if generated_ips:
    df[dataset.src_ip_col] = df[dataset.src_ip_col].apply(lambda x: socket.inet_ntoa(struct.pack('>I', random.randint(0xac100001, 0xac1f0001))))

In [50]:
if sort_timestamp:
    df[dataset.timestamp_col] = pd.to_datetime(df[dataset.timestamp_col].str.strip(), format=dataset.timestamp_format)
    df.sort_values(dataset.timestamp_col, inplace=True)

In [51]:
if use_port_in_address:
    df[dataset.src_port_col] = df[dataset.src_port_col].astype(float).astype(int).astype(str) # to remove the decimal point
    df[dataset.src_ip_col] = df[dataset.src_ip_col] + ':' + df[dataset.src_port_col]

    df[dataset.dst_port_col] = df[dataset.dst_port_col].astype(float).astype(int).astype(str) # to remove the decimal point
    df[dataset.dst_ip_col] = df[dataset.dst_ip_col] + ':' + df[dataset.dst_port_col]

In [52]:
df.head()

,Timestamp,Scr_IP,Scr_port,Des_IP,Des_port,Protocol,Service,Duration,Scr_bytes,Des_bytes,...,Login_attempt,Succesful_login,File_activity,Process_activity,read_write_physical.process,is_privileged,class1,class2,class3,Class
index,,,,,,,,,,,,,,,,,,,,,
0,2020-01-09 03:35:56,192.168.2.199,49278,192.168.2.10,80,1,4,75756,1849,13843,...,0,0,0,0,0,0,Scanning_vulnerability,Reconnaissance,1,6
1,2020-01-12 23:31:13,10.0.1.5,39769,131.236.3.92,53,2,2,83,16696,1,...,0,0,0,0,0,0,Normal,Normal,0,4
2,2020-01-08 22:28:06,172.24.1.80,59050,172.24.1.1,53,2,2,132,10806,14586,...,0,0,0,0,0,0,Normal,Normal,0,4
3,2020-02-26 22:54:00,192.168.2.196,37966,192.168.2.10,1880,1,16,260329,689,16960,...,1,1,1,1,1,1,Normal,Normal,0,4
4,2019-12-15 23:30:12,172.24.1.80,38233,172.24.1.1,53,2,2,74,0,0,...,0,0,0,0,0,0,Normal,Normal,0,4


In [53]:
if multi_class:
    y = df[dataset.class_num_col]
else:
    y = df[dataset.label_col]

if sort_timestamp:
    X_tr, X_test, y_tr, y_test = train_test_split(
        df, y, test_size=test_size)
    
    X_train, X_val, y_train, y_val = train_test_split(
        X_tr, y_tr, test_size=validation_size)
else:
    X_tr, X_test, y_tr, y_test = train_test_split(
        df, y, test_size=test_size, random_state=13, stratify=y)
    
    X_train, X_val, y_train, y_val = train_test_split(
        X_tr, y_tr, test_size=validation_size, random_state=13, stratify=y_tr)

del df

In [54]:
if graph_type == "line" and use_node_features:
    add_centralities(df = X_train, new_path=None, graph_path=None, dataset=dataset, cn_measures=cn_measures, network_features=network_features, create_using=nx.MultiDiGraph())
    add_centralities(df = X_val, new_path=None, graph_path=None, dataset=dataset, cn_measures=cn_measures, network_features=network_features, create_using=nx.MultiDiGraph())
    add_centralities(df = X_test, new_path=None, graph_path=None, dataset=dataset, cn_measures=cn_measures, network_features=network_features, create_using=nx.MultiDiGraph())
    cols_to_norm = list(set(cols_to_norm) | set(network_features))
    

In [55]:
scaler = StandardScaler()

X_train[cols_to_norm] = scaler.fit_transform(X_train[cols_to_norm])
X_train['h'] = X_train[ cols_to_norm ].values.tolist()

cols_to_drop = list(set(list(X_train.columns)) - set(list([dataset.label_col, dataset.src_ip_col, dataset.dst_ip_col, dataset.class_num_col, 'h'])))
X_train.drop(cols_to_drop, axis=1, inplace=True)

X_val[cols_to_norm] = scaler.transform(X_val[cols_to_norm])
X_val['h'] = X_val[ cols_to_norm ].values.tolist()
X_val.drop(cols_to_drop, axis=1, inplace=True)

X_test[cols_to_norm] = scaler.transform(X_test[cols_to_norm])
X_test['h'] = X_test[ cols_to_norm ].values.tolist()
X_test.drop(cols_to_drop, axis=1, inplace=True)

In [56]:
if graph_type == "window" or graph_type == "line":

    create_weightless_window_graph(
        df=X_train,
        dataset=dataset,
        window_size=window_size,
        line_graph=graph_type == "line",
        folder_path=os.path.join(folder_path, "training"),
        edge_attr= ['h', dataset.label_col, dataset.class_num_col],
        file_type="pkl")
    
    create_weightless_window_graph(
        df=X_val,
        dataset=dataset,
        window_size=window_size,
        line_graph=graph_type == "line",
        folder_path=os.path.join(folder_path, "validation"),
        edge_attr= ['h', dataset.label_col, dataset.class_num_col],
        file_type="pkl")
    
    create_weightless_window_graph(
        df=X_test,
        dataset=dataset,
        window_size=window_size,
        line_graph=graph_type == "line",
        folder_path=os.path.join(folder_path, "testing"),
        edge_attr= ['h', dataset.label_col, dataset.class_num_col],
        file_type="pkl")

In [57]:
if graph_type == "flow":
	os.makedirs(folder_path, exist_ok=True)
	print(f"==>> X_train.shape: {X_train.shape}")
	print(f"==>> X_val.shape: {X_val.shape}")
	print(f"==>> X_test.shape: {X_test.shape}")


==>> X_train.shape: (511768, 5)
==>> X_val.shape: (56864, 5)
==>> X_test.shape: (63182, 5)


In [58]:
if graph_type == "flow":
    graph_name = "training_graph"

    G = nx.from_pandas_edgelist(X_train, dataset.src_ip_col, dataset.dst_ip_col, ['h',dataset.label_col, dataset.class_num_col], create_using=nx.MultiDiGraph())
    
    if use_node_features:
        add_centralities_as_node_features(df=None, G=G, graph_path=None, dataset=dataset, cn_measures=cn_measures)
        
        for node in G.nodes():
            centralities = []
            for centrality in cn_measures:
                centralities.append(G.nodes[node].get(centrality, 0)) # Default to 0 if missing
                
                # Combine features into a single vector
            n_feats = np.array(centralities, dtype=np.float32)
            
            # Add the new feature to the node
            G.nodes[node]["n_feats"] = n_feats
            
    # get netowrk properties
    graph_measures = calculate_graph_measures(G, f"{folder_path}/{graph_name}_measures.json", verbose=True)
    print(f"==>> graph_measures: {graph_measures}")

    # graph_measures = calculate_graph_measures(nx.DiGraph(G), "datasets/" + name + "/training_graph_simple_measures.json", verbose=True)
    # print(f"==>> graph_measures: {graph_measures}")

    with open(f"{folder_path}/{graph_name}.pkl", "wb") as f:
        pickle.dump(G, f)

==>> calculated degrees, in 0.0001368001103401184 seconds
==>> graph_measures: {'number_of_nodes': 107, 'number_of_edges': 511768, 'max_degree': 238453, 'avg_degree': 9565.757009345794, 'density': 45.12149532710281}


In [59]:
if graph_type == "flow":
    graph_name = "validation_graph"

    G = nx.from_pandas_edgelist(X_val, dataset.src_ip_col, dataset.dst_ip_col, ['h',dataset.label_col, dataset.class_num_col], create_using=nx.MultiDiGraph())
    
    if use_node_features:
        add_centralities_as_node_features(df=None, G=G, graph_path=None, dataset=dataset, cn_measures=cn_measures)
        
        for node in G.nodes():
            centralities = []
            for centrality in cn_measures:
                centralities.append(G.nodes[node].get(centrality, 0)) # Default to 0 if missing
                
                # Combine features into a single vector
            n_feats = np.array(centralities, dtype=np.float32)
            
            # Add the new feature to the node
            G.nodes[node]["n_feats"] = n_feats
            
    # get netowrk properties
    graph_measures = calculate_graph_measures(G, f"{folder_path}/{graph_name}_measures.json", verbose=True)
    print(f"==>> graph_measures: {graph_measures}")

    # graph_measures = calculate_graph_measures(nx.DiGraph(G), "datasets/" + name + "/training_graph_simple_measures.json", verbose=True)
    # print(f"==>> graph_measures: {graph_measures}")

    with open(f"{folder_path}/{graph_name}.pkl", "wb") as f:
        pickle.dump(G, f)

==>> calculated degrees, in 0.00010430067777633667 seconds
==>> graph_measures: {'number_of_nodes': 78, 'number_of_edges': 56864, 'max_degree': 26533, 'avg_degree': 1458.051282051282, 'density': 9.467865467865467}


In [60]:
if graph_type == "flow":
    graph_name = "testing_graph"
    
    G = nx.from_pandas_edgelist(X_test, dataset.src_ip_col, dataset.dst_ip_col, ['h', dataset.label_col, dataset.class_num_col], create_using=nx.MultiDiGraph())
    
    if use_node_features:
        add_centralities_as_node_features(df=None, G=G, graph_path=None, dataset=dataset, cn_measures=cn_measures)
        
        for node in G.nodes():
            centralities = []
            for centrality in cn_measures:
                centralities.append(G.nodes[node].get(centrality, 0)) # Default to 0 if missing
                
                # Combine features into a single vector
            n_feats = np.array(centralities, dtype=np.float32)
            
            # Add the new feature to the node
            G.nodes[node]["n_feats"] = n_feats
            
    graph_measures = calculate_graph_measures(G, f"{folder_path}/{graph_name}_measures.json", verbose=True)
    print(f"==>> graph_measures: {graph_measures}")
    
    # graph_measures = calculate_graph_measures(nx.DiGraph(G_test), "datasets/" + name + "/testing_graph_simple_measures.json", verbose=True)
    # print(f"==>> graph_measures: {graph_measures}")
    
    with open(f"{folder_path}/{graph_name}.pkl", "wb") as f:
        pickle.dump(G, f)

==>> calculated degrees, in 0.00010609999299049377 seconds
==>> graph_measures: {'number_of_nodes': 80, 'number_of_edges': 63182, 'max_degree': 29310, 'avg_degree': 1579.55, 'density': 9.997151898734177}
